In [ ]:
import pandas as pd
file_path = "newcrest.xlsx"
df1 = pd.read_excel(file_path,'MainOffer',skiprows = 3)
#mainoffer
df1.head(10)

In [ ]:
##converseion

#customer pack qty: need to updated with [Cust UOM]
cus_qty_unique = df1['Cust UOM'].unique()
print("Unique customer type is:",cus_qty_unique)
#select item description, UoM, total, Part Description, price UoM, stock status

In [ ]:
import pandas as pd
import numpy as np
###---for BW one, always named after Price UOM---###
cus_uom,bw_uom = df1["Cust UOM"],df1["Price UOM"]

##iterate the column name
#for col in df1.columns:
 #   print(col)
    
###---"Packing""UoM""Price UOM" name might be different---###
df2 = df1[["Sort Key","Cust Category","Cust Description","Cust UOM","Part Description","Price UOM","Part Number"]]
##Assume all conversion is 1 currently
df2.loc[:, "conversion"] = 1.0
##all the Cust Pack Size & BW Pack Size = 1
df2.loc[:, "Cust_Size"] = 1.0
df2.loc[:, "BW_Size"] = 1.0
df2

In [ ]:
###convert for customer

###important Cell

import re
for index, row in df2.iterrows():
    cust_uom = 1
    Bw_qty = 1
    ##customer size conversion
    ##customer description   
    if pd.isna(row['Cust UOM']):
        continue
    if re.match(r'^([a-zA-Z]+)([0-9]+)$',row['Cust UOM']):
        ##get the number
        cust_uom = re.search(r'^([a-zA-Z]+)([0-9]+)$', row['Cust UOM']).group(2)
        
    elif re.match(r'^([0-9]+)([a-zA-Z]+)$',row['Cust UOM']):
        cust_uom = re.search(r'^([0-9]+)([a-zA-Z]+)$', row['Cust UOM']).group(1)
        
    elif re.search(r'(?i)(\d+)\s*X\s*(\d+)' , row['Cust UOM']):
        #6 X 1 Ltr
        cust_uom_left_num = re.search(r'(?i)(\d+)\s*X\s*(\d+)', row['Cust UOM']).group(1)
        #cust_uom_left_num = [int(word) for word in cust_uom_left.split() if word.isdigit()] if cust_uom_left else 1
        cust_uom_right_num = re.search(r'(?i)(\d+)\s*X\s*(\d+)', row['Cust UOM']).group(2)
        #cust_uom_right_num = [int(word) for word in cust_uom_right.split() if word.isdigit()] if cust_uom_right else 1
        tail  = re.search(r'(\d+)(\D*)$',(row["Cust Description"])).group(1) or re.search(r'(\D*)(\d+)$',(row["Cust Description"])).group(2)
        if (cust_uom_left_num or cust_uom_right_num) and tail and (int(tail) == cust_uom_left_num[0] or int(tail) == cust_uom_right_num[0] or int(tail) == cust_uom_left_num or int(tail) ==cust_uom_right_num):
            cust_uom = tail
        else:
            cust_uom = min(cust_uom_left_num[0],cust_uom_right_num[0])
        #update Bw_qty
        bracket = re.search(r'\((.*)\)' , row["Part Description"])
        Bw_qty = bracket if (bracket and bracket.group(1).isdigit()) else 1
        
    elif re.match(r'(?i)(\d+)(m|cm|g|kg|ml|l|SH|SHT|Ltr|Lt|ltr|LTRS|GM|)$',row['Cust UOM']):
        ###if BW UOM = each
        uom_match = re.match(r'(?i)(\d+)(m|cm|g|kg|ml|l|SH|SHT|Ltr|Lt|ltr|LTRS|GM|)$',row['Cust UOM'])
        if row['Price UOM'] in (["EA","Each","ea","EACH"]):
            cust_value =  uom_match.group(1)
            cust_unit =  uom_match.group(2)
            bw_search = re.search(r'(?i)(\d+)(m|cm|g|kg|ml|l|SH|SHT|Ltr|Lt|ltr|LTRS|GM|)$', row["Part Description"])
            if bw_search:
                bw_value = bw_search.group(1)
                bw_unit = bw_search.group(2)            
                if (cust_unit.lower() == bw_unit.lower()):
                    Bw_qty = bw_value
                    cust_uom = cust_value 
                elif (len(cust_unit) >= len(bw_unit) and cust_unit[0:len(bw_unit)].lower() == bw_unit.lower()) :
                    Bw_qty = bw_value
                    cust_uom = cust_value
                elif (len(cust_unit) < len(bw_unit) and bw_unit[0:len(cust_unit)].lower() == cust_unit.lower()):
                    Bw_qty = bw_value
                    cust_uom = cust_value
                ## g vs kg || m vs cm || L(LTR) vs ML 
                elif (cust_unit.lower() == 'g' and bw_unit.lower() == 'kg') or (cust_unit.lower() == 'm' and bw_unit.lower() == 'cm') or (cust_unit.lower() in (["l","ltr"]) and bw_unit.lower() == 'ml'):
                    if (cust_unit.lower() == 'm' and bw_unit.lower() == 'cm'):
                        Bw_qty = bw_value/100
                    else:
                        Bw_qty = bw_value/1000
                    cust_uom = cust_value
                elif (cust_unit.lower() == 'kg' and bw_unit.lower() == 'g') or (cust_unit.lower() == 'cm' and bw_unit.lower() == 'm') or (cust_unit.lower()  == "ml" and bw_unit.lower() in (["l","ltr"])):
                    if (cust_unit.lower() == 'cm' and bw_unit.lower() == 'm'):
                        cust_uom = cust_value/100
                    else:
                        cust_uom = cust_value/1000
                    Bw_qty = bw_value
            else: ###liter
                liter_search = re.search(r'(?i)(\d+)[l|LITRE|LT]', row["Part Description"])
                liter_unit = 'l'
                liter_value = liter_search.group(1) if liter_search else 1
                if (cust_unit.lower() == liter_unit.lower() or (len(cust_unit) >= len(liter_unit) and cust_unit[0:len(liter_unit)].lower() == liter_unit.lower()) or (len(cust_unit) < len(liter_unit) and liter_unit[0:len(cust_unit)].lower() == cust_unit.lower())):
                    Bw_qty = liter_value
                    cust_uom = cust_value
            
    else:
        if not (pd.isna(row['Cust Description'])): 
            cust_qty = re.search(r'\((.*)\)' , row['Cust Description']) 
            if cust_qty:  
                if (re.search(r'(\d+)\s*\.\s*(\d+)X(\w+)',cust_qty.group(1))):
                    num1 = re.match(r'(\d+)\s*\.\s*(\d+)X(\w+)',cust_qty.group(1)).group(1)
                    right = re.match(r'(\d+)\s*\.\s*(\d+)X(\w+)',cust_qty.group(1)).group(3)
                    num2 = re.match(r'^(\d+)',right).group(1)
                    if (int(num1) <= 0):
                        num1 = 1        
                    if (int(num2) <= 0):
                        num2 = 1
                    cust_uom = int(num1)*int(num2)
                    if cust_uom > 1000:
                        cust_uom = 1
                    
                elif re.match(r'(\d+)X(\w+)',cust_qty.group(1)):
                    num1 = re.match(r'(\d+)X(\w+)',cust_qty.group(1)).group(1)
                    right = re.match(r'(\d+)X(\w+)',cust_qty.group(1)).group(2)
                    num2 = re.match(r'^(\d+)',right).group(1) 
                    if (int(num1) <= 0):
                        num1 = 1        
                    if (int(num2) <= 0):
                        num2 = 1
                    cust_uom = int(num1)*int(num2)  
                    if cust_uom > 1000: 
                        cust_uom = 1
                elif (re.search(r'-',cust_qty.group(1))):
                    cust_uom = 1               
                elif (re.search(r'^\s*(\d+)\s*$',cust_qty.group(1))):
                    cust_uom = re.search(r'^\s*(\d+)\s*$',cust_qty.group(1)).group() 

                elif (re.search(r'(\w+)(\s*)=(\s*)(\d+)',cust_qty.group(1))):
                    #if BAG K/TIDY OSO WHITE 9UM 27L (ROLL=50) 50
                    cust_uom =  re.search(r'(\w+)(\s*)=(\s*)(\d+)',cust_qty.group(1)).group(4)

                ###of group
                elif (re.search(r'(?i)of',cust_qty.group(1))):
                    if (re.search(r'(?i)(\d+)\s*(Of)\s*(\d+)',cust_qty.group(1))):
                        of_qty = re.search(r'(?i)(\d+)\s*(Of)\s*(\d+)',cust_qty.group(1))
                        left = of_qty.group(1)
                        right = of_qty.group(3)
                        cust_uom = int(left)*int(right) 
                    elif (re.search(r'(?i)(\d+)\s*(of)',cust_qty.group(1))):
                        cust_uom = re.search(r'(?i)(\d+)\s*(of)',cust_qty.group(1)).group(1)
                    elif (re.search(r"(?i)of(\s*)(\d+)$",cust_qty.group(1))):
                        cust_uom = re.search(r"(?i)of(\s*)(\d+)$",cust_qty.group(1)).group(2)  
                elif (re.search(r"(?i)(\d+)(\s*)(pkt|pack|pk|box|bx|bag|b|bg|blk|set|carton|pads|pe)$",cust_qty.group(1))):
                    cust_uom = re.search(r"(?i)(\d+)(\s*)(pkt|pack|pk|box|bx|bag|b|bg|blk|set|carton|pads|pe)$",cust_qty.group(1)).group(1)  
            
                elif (re.search(r"(?i)(pkt|pack|pk|box|bx|bag|b|bg|blk|set|carton|pads|pe)(\s*)(\d+)$",cust_qty.group(1))):
                    cust_uom = re.search(r"(?i)(pkt|pack|pk|box|bx|bag|b|bg|blk|set|carton|pads|pe)(\s*)(\d+)$",cust_qty.group(1)).group(3) 

                elif (re.search(r"(?i)(pkt|pack|pk|box|bx|bag|b|bg|blk|set|carton|pads|pe)\/(\d+)",cust_qty.group(1))):
                    cust_uom = re.search(r"(?i)(pkt|pack|pk|box|bx|bag|b|bg|blk|set|carton|pads|pe)\/(\d+)",cust_qty.group(1)).group(2)

                elif (re.search(r"(?i)(\d+)\D*\/(pkt|pack|pk|box|bx|bag|b|bg|blk|set|carton|pads|pe)",cust_qty.group(1))):
                    cust_uom = re.search(r"(?i)(\d+)\D*\/(pkt|pack|pk|box|bx|bag|b|bg|blk|set|carton|pads|pe)",cust_qty.group(1)).group(1)

                elif (re.search(r'(\D*)(\d+)(\D*)$',cust_qty.group(1))):
                    cust_uom = re.search(r'(\D*)(\d+)(\D*)$',cust_qty.group(1)).group(2) 

            ## if there is only one bracket
            elif re.search(r'\((.*)' , row['Cust Description']) :           
            ## eg: PAPER TOILET UNI TORK RL 850SHT 1ST4 (48
                inner = re.search(r'\((.*)' , row['Cust Description']).group(1)
                if inner.isdigit():
                    cust_uom = inner 

            ###no bracket
            elif (re.search(r"(?i)(pkt|pack|pk|box|bx|bag|b|bg|blk|set|carton|pads|pe)\/(\d+)$", row['Cust Description'])): 
                ##eg:bag/10
                cust_uom = re.search(r"(?i)(pkt|pack|pk|box|bx|bag|b|bg|blk|set|carton|pads|pe)\/(\d+)$", row['Cust Description']).group(2)
            elif (re.search(r"(?i)(\d+)\D*\/(pkt|pack|pk|box|bx|bag|b|bg|blk|set|carton|pads|pe)", row['Cust Description'])): 
                #2/pk
                cust_uom = re.search(r"(?i)(\d+)\D*\/(pkt|pack|pk|box|bx|bag|b|bg|blk|set|carton|pads|pe)", row['Cust Description']).group(1)
            elif (re.search(r"(?i)(\d+)(\s*)(pkt|pack|pk|box|bx|bag|b|bg|blk|set|carton|pads|pe)$",row["Cust Description"])):
                ##3PK
                cust_uom = re.search(r"(?i)(\d+)(\s*)(pkt|pack|pk|box|bx|bag|b|bg|blk|set|carton|pads|pe)$",row["Cust Description"]).group(1) 
            elif (re.search(r"(?i)(pkt|pack|pk|box|bx|bag|b|bg|blk|set|carton|pads|pe)(\s*)(\d+)", row['Cust Description'])): 
                #Pack 100
                cust_uom = re.search(r"(?i)(pkt|pack|pk|box|bx|bag|b|bg|blk|set|carton|pads|pe)(\s*)(\d+)", row['Cust Description']).group(3)
            elif (re.search(r'(\w+)(\s*)=(\s*)(\d+)',row['Cust Description'])):
                cust_uom = re.search(r'(\w+)(\s*)=(\s*)(\d+)',row['Cust Description']).group(4)                             
        else:
            cust_uom = 1 
        
    ###transfer of BW UOM
    if not (pd.isna(row['Part Description'])): 
        if (pd.isna(row['Price UOM'])):
            continue
        elif row['Price UOM'] in (["100","1000"]):
            Bw_qty = int(row['Price UOM'])  
        else:
            Bw_unit = re.search(r'\((.*)\)' , row['Part Description']) 
            if Bw_unit:  
                if (re.search(r'(\d+)\s*\.\s*(\d+)X(\w+)',Bw_unit.group(1))):
                    Bw_num1 = re.match(r'(\d+)\s*\.\s*(\d+)X(\w+)',Bw_unit.group(1)).group(1)
                    right = re.match(r'(\d+)\s*\.\s*(\d+)X(\w+)',Bw_unit.group(1)).group(3)
                    Bw_num2 = re.match(r'^(\d+)',right).group(1)
                    if (int(Bw_num1) <= 0):
                        Bw_num1 = 1        
                    if (int(Bw_num2) <= 0):
                        Bw_num2 = 1
                    Bw_qty = int(Bw_num1)*int(Bw_num2)
                    if Bw_qty > 1000:
                        Bw_qty = 1
                elif re.match(r'(\d+)X(\w+)',Bw_unit.group(1)):
                    Bw_num1 = re.match(r'(\d+)X(\w+)',Bw_unit.group(1)).group(1)
                    right = re.match(r'(\d+)X(\w+)',Bw_unit.group(1)).group(2)
                    Bw_num2 = re.match(r'^(\d+)',right).group(1) 
                    if (int(Bw_num1) <= 0):
                        Bw_num1 = 1        
                    if (int(Bw_num2) <= 0):
                        Bw_num2 = 1
                    Bw_qty = int(Bw_num1)*int(Bw_num2)  
                    if Bw_qty > 1000:
                        Bw_qty = 1
                elif (re.search(r'-',Bw_unit.group(1))):
                    Bw_qty = 1               
                elif (re.search(r'^\s*(\d+)\s*$',Bw_unit.group(1))):
                    Bw_qty = re.search(r'^\s*(\d+)\s*$',Bw_unit.group(1)).group() 
                elif (re.search(r'(\w+)(\s*)=(\s*)(\d+)',Bw_unit.group(1))):
                    #if BAG K/TIDY OSO WHITE 9UM 27L (ROLL=50) 50
                    Bw_qty =  re.search(r'(\w+)(\s*)=(\s*)(\d+)',Bw_unit.group(1)).group(4)

                ###of group
                elif (re.search(r'(?i)of',Bw_unit.group(1))):
                    if (re.search(r'(?i)(\d+)\s*(of)\s*(\d+)',Bw_unit.group(1))):
                        of_qty = re.search(r'(?i)(\d+)\s*(of)\s*(\d+)',Bw_unit.group(1))
                        left = of_qty.group(1)
                        right = of_qty.group(3)
                        Bw_qty = int(left)*int(right) 
                    elif (re.search(r'(?i)(\d+)\s*(of)',Bw_unit.group(1))):
                        Bw_qty = re.search(r'(?i)(\d+)\s*(of)',Bw_unit.group(1)).group(1)
                    elif (re.search(r"(?i)of(\s*)(\d+)$",Bw_unit.group(1))):
                        Bw_qty = re.search(r"(?i)of(\s*)(\d+)$",Bw_unit.group(1)).group(2) 

                
                elif (re.search(r"(?i)(\d+)(\s*)(pkt|pack|pk|box|bx|bag|b|bg|blk|set|carton|pads|pe)$",Bw_unit.group(1))):
                        Bw_qty = re.search(r"(?i)(\d+)(\s*)(pkt|pack|pk|box|bx|bag|b|bg|blk|set|carton|pads|pe)$",Bw_unit.group(1)).group(1)  

                elif (re.search(r"(?i)(pkt|pack|pk|box|bx|bag|b|bg|blk|set|carton|pads|pe)(\s*)(\d+)$",Bw_unit.group(1))):
                    Bw_qty = re.search(r"(?i)(pkt|pack|pk|box|bx|bag|b|bg|blk|set|carton|pads|pe)(\s*)(\d+)$",Bw_unit.group(1)).group(3) 

                elif (re.search(r"(?i)(pkt|pack|pk|box|bx|bag|b|bg|blk|set|carton|pads|pe)\/(\d+)",Bw_unit.group(1))):
                    Bw_qty = re.search(r"(?i)(pkt|pack|pk|box|bx|bag|b|bg|blk|set|carton|pads|pe)\/(\d+)",Bw_unit.group(1)).group(2)

                elif (re.search(r"(?i)(\d+)\D*\/(pkt|pack|pk|box|bx|bag|b|bg|blk|set|carton|pads|pe)",Bw_unit.group(1))):
                    Bw_qty = re.search(r"(?i)(\d+)\D*\/(pkt|pack|pk|box|bx|bag|b|bg|blk|set|carton|pads|pe)",Bw_unit.group(1)).group(1)

                elif (re.search(r'(\D*)(\d+)(\D*)$',Bw_unit.group(1))):
                    Bw_qty = re.search(r'(\D*)(\d+)(\D*)$',Bw_unit.group(1)).group(2)  
                
            ## if there is only one bracket
            elif re.search(r'\((.*)' , row['Part Description']) :           
            ## eg: PAPER TOILET UNI TORK RL 850SHT 1ST4 (48
                inner = re.search(r'\((.*)' , row['Part Description']).group(1)
                if inner.isdigit():
                    Bw_qty = inner 

            ###no bracket
            elif (re.search(r"(?i)(pkt|pack|pk|box|bx|bag|b|bg|blk|set|carton|pads|pe)\/(\d+)$", row['Part Description'])): 
                ##eg:bag/10
                Bw_qty = re.search(r"(?i)(pkt|pack|pk|box|bx|bag|b|bg|blk|set|carton|pads|pe)\/(\d+)$", row['Part Description']).group(2)
            elif (re.search(r"(?i)(\d+)\D*\/(pkt|pack|pk|box|bx|bag|b|bg|blk|set|carton|pads|pe)", row['Part Description'])): 
                #2/pk
                Bw_qty = re.search(r"(?i)(\d+)\D*\/(pkt|pack|pk|box|bx|bag|b|bg|blk|set|carton|pads|pe)", row['Part Description']).group(1)
            elif (re.search(r"(?i)(\d+)(\s*)(pkt|pack|pk|box|bx|bag|b|bg|blk|set|carton|pads|pe)$",row["Part Description"])):
                ##3PK
                Bw_qty = re.search(r"(?i)(\d+)(\s*)(pkt|pack|pk|box|bx|bag|b|bg|blk|set|carton|pads|pe)$",row["Part Description"]).group(1) 
            elif (re.search(r"(?i)(pkt|pack|pk|box|bx|bag|b|bg|blk|set|carton|pads|pe)(\s*)(\d+)", row['Part Description'])): 
                #Pack 100
                Bw_qty = re.search(r"(?i)(pkt|pack|pk|box|bx|bag|b|bg|blk|set|carton|pads|pe)(\s*)(\d+)", row['Part Description']).group(3)
            elif (re.search(r'(\w+)(\s*)=(\s*)(\d+)',row['Part Description'])):
                Bw_qty = re.search(r'(\w+)(\s*)=(\s*)(\d+)',row['Part Description']).group(4)        

    else: #no description
        if row['Price UOM'] in (["100","1000"]):
            Bw_qty = int(row['Price UOM'])
        else:
            Bw_qty = 1     
             
    if int(cust_uom) == 0:
         cust_uom = 1
    if int(Bw_qty) == 0:
        Bw_qty = 1
        
    ##update with the customer and BW size          
    df2.at[index, 'Cust_Size'] = cust_uom
    df2.at[index, 'BW_Size'] = Bw_qty
    ##update with the conversion      
    df2.at[index, 'conversion'] = df2.at[index, 'Cust_Size']/df2.at[index, 'BW_Size']

In [ ]:
df2.to_csv (r'newcrest_converesion_v2.csv', index = False, header=True)